In [1]:
import tensorflow as tf
from tensorflow.contrib import learn
from sklearn import datasets, metrics, preprocessing
boston = datasets.load_boston()
x_data = preprocessing.StandardScaler().fit_transform(boston.data)
y_data = boston.target

c:\users\seo\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
feature_columns = learn.infer_real_valued_columns_from_input(x_data)
reg = learn.LinearRegressor(
feature_columns=feature_columns,
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1))
reg.fit(x_data,boston.target,steps=100,batch_size=8)
MSE = reg.evaluate(x_data,boston.target,steps=1)
print("MSE: ",MSE)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000001811B278>, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_environment': 'local', '_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_master': '', '_tf_random_seed': None, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_task_type': None, '_task_id': 0, '_model_dir': 'C:\\Users\\Seo\\AppData\\Local\\Temp\\tmpgb_zam7b', '_num_ps_replicas': 0, '_evaluation_master': ''}
Instructions for updating:
Please switch to the Estimator interface.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimat

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
data = input_data.read_data_sets('/data/',one_hot=False)
x_data, y_data = data.train.images,data.train.labels.astype(np.int32)
x_test, y_test = data.test.images,data.test.labels.astype(np.int32)

num_steps = 20000
minibatch_size = 128

feature_columns = learn.infer_real_valued_columns_from_input(x_data)

dnn = learn.DNNClassifier(
feature_columns = feature_columns,
hidden_units = [200],
n_classes = 10,
optimizer = tf.train.AdamOptimizer(0.2))

dnn.fit(x=x_data,y=y_data,steps=num_steps,batch_size=minibatch_size)

test_acc = dnn.evaluate(x=x_test,y=y_test,steps=1)["accuracy"]
print("Test accuracy : ",test_acc)

Extracting /data/train-images-idx3-ubyte.gz
Extracting /data/train-labels-idx1-ubyte.gz
Extracting /data/t10k-images-idx3-ubyte.gz
Extracting /data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000020121320>, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_environment': 'local', '_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_master': '', '_tf_random_seed': None, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_task_type': None, '_task_id': 0, '_model_dir': 'C:\\Users\\Seo\\AppData\\Local\\Temp\\tmp8psqzteh', '_num_ps_replicas': 0, '_evaluation_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
IN

In [9]:
from sklearn.metrics import confusion_matrix
y_pred = dnn.predict(x=x_test,as_iterable=False)
class_names = ['0','1','2','3','4','5','6','7','8','9']
cnf_matrix=confusion_matrix(y_test,y_pred)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available

In [ ]:
from tensorflow.contrib import layers
def model_fn(x,target,mode,params):
    y_ = tf.cast(target,tf.float32)
    x_image = tf.reshape(x,[-1,28,28,1])
    
    conv1 = layers.convolution2d(x_image,32,[5,5],
                                activation_fn=tf.nn.relu,
                                biases_initializer= tf.constant_initializer(0.1),
                                weights_initializer = tf.truncated_normal_initializer(stddev=0.1))
    pool1 = layers.max_pool2d(conv1,[2,2])
    
    conv2 = layers.convolution2d(pool1,64,[5,5],
                                activation_fn=tf.nn.relu,
                                biases_initializer= tf.constant_initializer(0.1),
                                weights_initializer = tf.truncated_normal_initializer(stddev=0.1))
    pool2 = layers.max_pool2d(conv2,[2,2])
    
    pool2_flat = tf.reshape(pool2,[-1,7*7*64])
    fc1 = layers.fully_connected(pool2_flat,1024,
                                activation_fn=tf.nn.relu,
                                biases_initializer =tf.constant_initializer(0.1),
                                weights_initializer = tf.truncated_normal_initializer(stddev=0.1))
    fc1_drop = layers.dropout(fc1,keep_prob=params['dropout'],
                             is_training=(mode=='train'))
    
    y_conv = layers.fully_connected(fc1_drop,10,activation_fn=None)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv,labels=y_))
    train_op = tf.contrib.layers.optimize_loss(
    loss=cross_entropy,
    global_step = tf.contrib.framework.get_global_step(),
    learning_rate=params['learning_rate'],
    optimizer='Adam')
    
    predictions = tf.argmax(y_conv,1)
    return predictions,cross_entropy,train_op

data = input_data.read_data_sets('/data/',one_hot=True)
x_data, y_data = data.train.images, np.int32(data.train.labels)
tf.cast(y_data,tf.float32)
tf.cast(y_data,tf.float32)

model_params={'learning_rate':1e-4,'dropout':0.5}
CNN = tf.contrib.learn.Estimator(model_fn=model_fn,params=model_params)

print("Starting training for %s steps max " % 5000)
CNN.fit(x=data.train.images,y=data.train.labels,batch_size=50,max_steps=5000)

test_acc=0

for ii in range(5):
    batch = data.test.next_batch(2000)
    predictions = list(CNN.predict(batch[0],as_iterable=True))
    test_acc = test_acc + (np.argmax(batch[1],1)==predictions).mean()
    
print(test_acc/5)

Extracting /data/train-images-idx3-ubyte.gz
Extracting /data/train-labels-idx1-ubyte.gz
Extracting /data/t10k-images-idx3-ubyte.gz
Extracting /data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000000200EBFD0>, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_environment': 'local', '_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_master': '', '_tf_random_seed': None, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_task_type': None, '_task_id': 0, '_model_dir': 'C:\\Users\\Seo\\AppData\\Local\\Temp\\tmpmj1bitdb', '_num_ps_replicas': 0, '_evaluation_master': ''}
Starting training for 5000 steps max 
INFO:tensorflow:Create CheckpointSaverHook.
I